# 🍎 Prompt Engineering Techniques with AIProjectClient 🍏

En este cuaderno, demostraremos algunas técnisa de `Prompt Engineering` utilizando el SDK **Azure AI Foundry**. Combinaremos los paquetes **`azure-ai-projects`** y **`azure-ai-inference`** para:

1. **Inicializar** un `AIProjectClient`.
2. **Obtener** un cliente de Chat Completions para realizar llamadas directas al LLM.
3. **Utilizar** diferentes **plantillas de prompt** para añadir el contexto del sistema.
4. **Enviar** mensajes de usuario con una temática de salud y fitness.

## 1. Initial Setup

Cargar variables de entorno, crear un `AIProjectClient`, y obtener un `ChatCompletionsClient`. También definiremos un prompt template para mostrar cómo se podría estructurar un system message.

In [18]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import DefaultAzureCredential
from azure.ai.inference.prompts import PromptTemplate
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, SystemMessage  # for chat messages

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
model_deployment = 'gpt-4.1'
load_dotenv(parent_dir / '../.env', override=True)

# Retrieve from environment
connection_string = os.environ.get("PROJECT_CONNECTION_STRING")
model_deployment = 'gpt-4o'

try:
    # Create the project client
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=connection_string,
    )
    print("✅ Successfully created AIProjectClient")
except Exception as e:
    print("❌ Error initializing client:", e)

✅ Successfully created AIProjectClient


### 1. Role Prompting

El role prompting es una técnica poderosa en prompt engineering que permite guiar a los modelos de IA para adoptar personalidades o especializaciones específicas. Este enfoque puede mejorar significativamente la calidad y relevancia de las respuestas generadas por la IA, haciendo que sean más adecuadas para tareas o dominios específicos.


In [5]:
# We'll define a function that runs chat completions with a system prompt & user prompt
roles = [
    ("Asesor Financiero", "Eres un asesor financiero experimentado en un banco. Explica los beneficios y los riesgos potenciales asociados con:"),
    ("Oficial de Préstamos", "Eres un oficial de préstamos con experiencia en un banco. Explica los pasos clave y los requisitos necesarios para:"),
    ("Representante de Servicio al Cliente", "Eres un representante de servicio al cliente amable en un banco. Proporciona una explicación clara y accesible de:")
]

topic = "El crédito hipotecario"

def chat_with_assistant(system_text: str):
    """Use chat completions to get a response from our LLM, with system instructions."""
    # Our system message template
    # We'll open the chat completions client
    with project_client.inference.get_chat_completions_client() as chat_client:
        # Construct messages: system + user
        system_message = SystemMessage(content=system_text)
        user_message = UserMessage(content=topic)

        # Send the request
        response = chat_client.complete(
            model=model_deployment,
            messages=[system_message, user_message]
        )

        return response.choices[0].message.content

In [6]:
for role, system_text in roles:
    print(f"\n--- Explicacion {role} ---\n")
    response = chat_with_assistant(system_text)
    print(response)
    print("-" * 50)


--- Explicacion Asesor Financiero ---

### Beneficios del Crédito Hipotecario

1. **Acceso a la Propiedad de Vivienda**:
   El beneficio más evidente es que un crédito hipotecario permite comprar una vivienda, incluso cuando no se dispone del capital necesario de forma inmediata. Esto puede ser especialmente importante en mercados inmobiliarios donde los precios son altos.

2. **Construcción de Patrimonio**:
   Con el tiempo, los pagos regulares del crédito hipotecario incrementan la parte de la vivienda que se posee, lo que contribuye a la construcción del patrimonio personal.

3. **Potenciales Ventajas Fiscales**:
   En algunos países, los intereses pagados en un crédito hipotecario pueden ser deducibles de impuestos, lo que puede resultar en ahorros fiscales.

4. **Tasas de Interés Generalmente Más Bajas**:
   Los créditos hipotecarios suelen tener tasas de interés más bajas en comparación con otros tipos de préstamos debido a que son respaldados por un bien inmueble como garantía.

### 2. Output Formatting

Una longitud y un formato de salida consistentes son cruciales para la experiencia del usuario y la claridad de la interfaz. Las restricciones ayudan a mantener esta consistencia, evitando una variabilidad abrumadora en el texto generado.

In [23]:
structured_output_prompt = PromptTemplate.from_string(prompt_template="""
    system:
    Eres un experto en banca. 
    Extrae la evaluación crediticia del cliente, los productos financieros recomendados y el plan de acción a futuro a partir del siguiente texto.
    Devuelve la respuesta en formato JSON dentro ```json ``` usando las claves "perfil_crediticio", "productos_recomendados" y "plan_de_accion".
                                             
    user:
    Texto: {{input_text}}                                        
    """
)

text = """
El cliente, Juan Pérez, de 40 años, ha mantenido en los últimos 5 años un comportamiento financiero ejemplar. Su historial crediticio es intachable, sin registros de morosidad y con pagos puntuales en todas sus cuentas. 
Se recomienda evaluar productos financieros como líneas de crédito, tarjetas de crédito con beneficios exclusivos y opciones de inversión en fondos de bajo riesgo. 
Como plan de acción, se sugiere implementar un programa de asesoría financiera personalizada que incluya revisiones trimestrales del portafolio y estrategias para optimizar su capacidad de endeudamiento.
"""

with project_client.inference.get_chat_completions_client() as chat_client:
        # Create the chain using the prompt and model
        messages = structured_output_prompt.create_messages(
            input_text=text
        )
        response = chat_client.complete(
            messages=messages,
            model=model_deployment,
        )

        print("\n--- Estructura de salida ---\n")
        print("Texto de entrada:", text)
        json_str = response.choices[0].message.content
        json_str = json_str.replace("```json", "").replace("```", "").strip()
        try:
            import json
            json_obj = json.loads(json_str)
            print("JSON:", json_obj)
        except json.JSONDecodeError as e:   
            print("Error decoding JSON:", e)
            print("Raw response:", json_str)


--- Estructura de salida ---

Texto de entrada: 
El cliente, Juan Pérez, de 40 años, ha mantenido en los últimos 5 años un comportamiento financiero ejemplar. Su historial crediticio es intachable, sin registros de morosidad y con pagos puntuales en todas sus cuentas. 
Se recomienda evaluar productos financieros como líneas de crédito, tarjetas de crédito con beneficios exclusivos y opciones de inversión en fondos de bajo riesgo. 
Como plan de acción, se sugiere implementar un programa de asesoría financiera personalizada que incluya revisiones trimestrales del portafolio y estrategias para optimizar su capacidad de endeudamiento.

JSON: {'perfil_crediticio': 'intachable, sin registros de morosidad y con pagos puntuales en todas sus cuentas', 'productos_recomendados': ['líneas de crédito', 'tarjetas de crédito con beneficios exclusivos', 'opciones de inversión en fondos de bajo riesgo'], 'plan_de_accion': 'implementar un programa de asesoría financiera personalizada que incluya revisi

### 3. Chain-of-Thought (CoT) Prompting

A medida que los modelos de lenguaje de IA se vuelven más avanzados, existe una necesidad creciente de guiarlos hacia la producción de salidas más transparentes, lógicas y verificables. La técnica de CoT prompting aborda esta necesidad al incentivar a los modelos a mostrar su proceso de trabajo, de manera similar a cómo los humanos abordan tareas complejas de resolución de problemas. Esta técnica no solo mejora la precisión de las respuestas de la IA, sino que también las hace más interpretables y confiables.

In [ ]:

standard_prompt = PromptTemplate.from_string(prompt_template="""
    system:
    Responde a la pregunta de manera clara y concisa.
                                             
    user:
    Pregunta: {{question}}                                        
    """
)

advanced_cot_prompt = PromptTemplate.from_string(prompt_template="""
    system:                                        
    Resuelve el siguiente problema paso a paso. Para cada paso:
    1. Indica lo que vas a calcular
    2. Escribe la fórmula que utilizarás (si aplica)
    3. Realiza el cálculo
    4. Explica el resultado
                                             
    user:
    Pregunta: {{question}}
"""
)

challenging_question = """
Un cliente de un banco deposita $10,000 en una cuenta con una tasa de interés anual del 5% compuesta mensualmente.
Adicionalmente, aporta $200 al final de cada mes.
Determine cuánto tiempo tomará que el saldo de la cuenta alcance los $20,000.
Exprese su respuesta en años y meses, redondeando al mes más cercano.
"""

def create_response(prompt_template: PromptTemplate, question: str):
    """Create a chain of thought using the provided prompt and model."""
    with project_client.inference.get_chat_completions_client() as chat_client:
        # Create the chain using the prompt and model
        messages = prompt_template.create_messages(
            question=question
        )
        response = chat_client.complete(
            messages=messages,
            model=model_deployment,
        )

        return response.choices[0].message.content


standard_response = create_response(standard_prompt, challenging_question)
cot_response =  create_response(advanced_cot_prompt, challenging_question)

print("Respuesta Estándar:")
print(standard_response)
print("\nRespuesta de COT:")
print(cot_response)

Respuesta Estándar:
Para resolver este problema, usaremos la fórmula del valor futuro de una serie de pagos con interés compuesto mensual. La fórmula para calcular el saldo futuro \( A \) de una cuenta con depósitos regulares es:

\[ A = P \left(1 + \frac{r}{n}\right)^{nt} + \frac{PMT \left(\left(1 + \frac{r}{n}\right)^{nt} - 1\right)}{\frac{r}{n}} \]

donde:
- \( P \) es el depósito inicial ($10,000).
- \( PMT \) es el depósito mensual recurrente ($200).
- \( r \) es la tasa de interés anual (0.05).
- \( n \) es el número de períodos de capitalización por año (12).
- \( t \) es el número de años.
- \( A \) es el saldo futuro deseado ($20,000).

Reorganizando para resolver \( t \) con los valores dados:

\[ 20,000 = 10,000 \left(1 + \frac{0.05}{12}\right)^{12t} + \frac{200 \left(\left(1 + \frac{0.05}{12}\right)^{12t} - 1\right)}{\frac{0.05}{12}} \]

Para hallar el tiempo \( t \), es más práctico usar un enfoque iterativo o una calculadora financiera o una hoja de cálculo debido a la na

### 4. Few-Shot Prompting

El aprendizaje automático tradicional a menudo requiere conjuntos de datos grandes para el entrenamiento, lo que puede ser un proceso que consume mucho tiempo y recursos. El Few-Shot Learning y el In-Context Learning abordan esta limitación aprovechando el poder de los grandes modelos de lenguaje para realizar tareas con solo unos pocos ejemplos. Este enfoque es particularmente valioso en escenarios donde los datos etiquetados son escasos o costosos de obtener.


In [16]:
few_shot_prompt_template = PromptTemplate.from_string(prompt_template="""
        Clasifica el sentimiento como Positivo, Negativo o Neutral.

        Ejemplos:
        Texto: La atención en este banco es excepcional.
        Sentimiento: Positivo

        Texto: Las comisiones de este banco son abusivas.
        Sentimiento: Negativo

        Texto: El servicio del cajero automático es regular.
        Sentimiento: Neutral

        Ahora, clasifica lo siguiente:
        Texto:{{input_text}}
"""
)

examples = [
        (
                "El servicio en la sucursal fue tan excepcional que me sentí como en un resort exclusivo, donde cada detalle fue cuidado y la atención personalizada hizo la diferencia.",
                "Positivo"
        ),
        (
                "La espera en el banco fue interminable y la información proporcionada resultó tan confusa que terminé sintiendo frustración y desilusión, lo cual empañó toda la experiencia.",
                "Negativo"
        ),
        (
                "El banco presentó una experiencia balanceada: se notaron esfuerzos innovadores, pero algunos aspectos del servicio dejaron pasar oportunidades de mejora, generando una sensación de regularidad.",
                "Neutral"
        ),
        (
                "La atención del personal fue creativa y proactiva, ofreciendo soluciones personalizadas y sugerencias innovadoras que realmente superaron mis expectativas.",
                "Positivo"
        ),
        (
                "Aunque la interacción fue amable, la falta de seguimiento detallado en mis consultas provocó cierta incertidumbre y una sensación de que el servicio podría ser más completo en futuras ocasiones.",
                "Neutral"
        )
]


with project_client.inference.get_chat_completions_client() as chat_client:
        for example in examples:
                # Create the chain using the prompt and model
                messages = few_shot_prompt_template.create_messages(
                        input_text=example[0]
                )
                response = chat_client.complete(
                        messages=messages,
                        model=model_deployment,
                )

                print(f"Texto: {example[0]}")
                print(f"Sentimiento: {response.choices[0].message.content}")
                print("-" * 50)


Texto: El servicio en la sucursal fue tan excepcional que me sentí como en un resort exclusivo, donde cada detalle fue cuidado y la atención personalizada hizo la diferencia.
Sentimiento: Sentimiento: Positivo
--------------------------------------------------
Texto: La espera en el banco fue interminable y la información proporcionada resultó tan confusa que terminé sintiendo frustración y desilusión, lo cual empañó toda la experiencia.
Sentimiento: Sentimiento: Negativo
--------------------------------------------------
Texto: El banco presentó una experiencia balanceada: se notaron esfuerzos innovadores, pero algunos aspectos del servicio dejaron pasar oportunidades de mejora, generando una sensación de regularidad.
Sentimiento: Sentimiento: Neutral
--------------------------------------------------
Texto: La atención del personal fue creativa y proactiva, ofreciendo soluciones personalizadas y sugerencias innovadoras que realmente superaron mis expectativas.
Sentimiento: Sentimient